<a href="https://colab.research.google.com/github/hubmapconsortium/ccf-linkml/blob/main/ASCT%2BB_Normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rdflib

In [2]:
import requests

def request_get(url, headers=None):
    """
    Performs a get request that provides a (somewhat) useful error message.
    """
    try:
        response = requests.get(url, headers)
    except ImportError:
        raise ImportError("Couldn't retrieve the data, check your URL")
    else:
        return response

def get_json(url):
    """Returns request in JSON (dict) format"""
    headers = {"Content-Type": "application/json; charset=utf-8"}
    return request_get(url, headers).json() 

/usr/local/Cellar/jupyterlab/3.2.8/libexec/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
class TableDataResultParser:
  """
  Represents the TableDataResult dictionary from the ASCT+B API endpoint
  """
  def get_data(self, o):
    return o['data']
  
  def get_metadata(self, o):
    return o['metadata']

  def get_row_number(self, o):
    return o['rowNumber']

  def get_anatomical_structures(self, o):
    return o['anatomical_structures']

  def get_cell_types(self, o):
    return o['cell_types']

  def get_biomarkers(self, o, type=None):
    key = "biomarkers"
    if type:
      key = f"biomarkers_{type}"
    return o[key]

  def get_references(self, o):
    return o['references']

  def get_id(self, o):
    return o['id']

  def get_label(self, o):
    return o['rdfs_label']

  def get_name(self, o):
    return o['name']

  def get_doi(self, o):
    return o['doi']

  def get_data_doi(self, o):
    return o['data_doi']

In [19]:
import re

from rdflib.graph import Graph, Namespace, URIRef, Literal, BNode, Collection
from rdflib.namespace import OWL, RDF, RDFS, XSD, DCTERMS

class AsctbDataTransformer:
  """
  Represents the algorithm to transform ASCT+B data to RDF
  """

  CCF_PREFIX = "http://purl.org/ccf/"
  FMA_PREFIX = "http://purl.org/sig/ont/fma/fma/"
  OBO_PREFIX = "http://purl.obolibrary.org/obo/"
  HGNC_PREFIX = "http://identifiers.org/hgnc/"
  ASCTB_TEMP_PREFIX = "https://purl.org/ccf/ASCTB-TEMP_"
  DOI_PREFIX = "http://doi.org/"

  CCF = Namespace(CCF_PREFIX)
  FMA = Namespace(FMA_PREFIX)
  OBO = Namespace(OBO_PREFIX)
  HGNC = Namespace(HGNC_PREFIX)
  ASCTB_TEMP = Namespace(ASCTB_TEMP_PREFIX)
  DOI = Namespace(DOI_PREFIX)

  def __init__(self, parser):
    self.graph = Graph()
    self.parser = parser

  @staticmethod
  def new(parser=TableDataResultParser()):
    return AsctbDataTransformer(parser)

  def handle_concept_id(self, obj):
    concept_id = self.parser.get_id(obj)
    term = None
    if concept_id:
      term = URIRef(self.expand_id(concept_id))
    return term

  def to_rdf(self, json):
    """
    Transforms ASCT+B data from format JSON to RDF.
    """
    self.graph.bind("ccf", self.CCF)
    self.graph.bind("fma", self.FMA)
    self.graph.bind("obo", self.OBO)
    self.graph.bind("hgnc", self.HGNC)
    self.graph.bind("asctb-temp", self.ASCTB_TEMP)
    self.graph.bind("doi", self.DOI)
    
    data = self.parser.get_data(json)
    metadata = self.parser.get_metadata(json)

    table_doi = self.parser.get_data_doi(metadata)
    table = URIRef(table_doi)
    
    self.graph.add((table, RDF.type, self.CCF.AsctbTable))

    record_list_node = BNode()
    record_list = []
    for row in data:
      record = BNode()
      self.graph.add((record, RDF.type, self.CCF.AsctbRecord))

      as_list = self.create_rdf_list(
          [self.parser.get_id(ast)
              for ast in self.parser.get_anatomical_structures(row)])
      self.graph.add((record, self.CCF.contains_anatomical_structures, as_list))

      ct_list = self.create_rdf_list(
          [self.parser.get_id(ct)
              for ct in self.parser.get_cell_types(row)])
      self.graph.add((record, self.CCF.contains_cell_types, ct_list))

      bm_list = self.create_rdf_list(
          [self.parser.get_id(bm)
              for bm in self.parser.get_biomarkers(row)])
      self.graph.add((record, self.CCF.contains_biomarkers, bm_list))

      bm_gene_list = self.create_rdf_list(
          [self.parser.get_id(bm)
              for bm in self.parser.get_biomarkers(row, "gene")])
      self.graph.add((record, self.CCF.contains_gene_biomarkers, bm_gene_list))

      bm_protein_list = self.create_rdf_list(
          [self.parser.get_id(bm)
              for bm in self.parser.get_biomarkers(row, "protein")])
      self.graph.add((record, self.CCF.contains_protein_biomarkers, bm_protein_list))

      bm_lipids_list = self.create_rdf_list(
          [self.parser.get_id(bm)
              for bm in self.parser.get_biomarkers(row, "lipids")])
      self.graph.add((record, self.CCF.contains_lipids_biomarkers, bm_lipids_list))

      bm_meta_list = self.create_rdf_list(
          [self.parser.get_id(bm)
              for bm in self.parser.get_biomarkers(row, "meta")])
      self.graph.add((record, self.CCF.contains_meta_biomarkers, bm_meta_list))

      bm_prot_list = self.create_rdf_list(
          [self.parser.get_id(bm)
              for bm in self.parser.get_biomarkers(row, "prot") if bm])
      self.graph.add((record, self.CCF.contains_prot_biomarkers, bm_prot_list))

      ref_list = self.create_rdf_list(
          [self.parser.get_doi(ref) 
              for ref in self.parser.get_references(row) if ref])
      self.graph.add((record, self.CCF.contains_references, ref_list))
      
      record_list.append(record)
    
    Collection(self.graph, record_list_node, record_list)
    self.graph.add((table, self.CCF.contains_records, record_list_node))

    return self.graph.serialize(format="turtle")

  def create_rdf_list(self, id_list):
    list_node = BNode()
    list_members = [URIRef(self.expand_id(id)) for id in id_list if id]
    Collection(self.graph, list_node, list_members)
    return list_node

  def expand_id(self, str):
    if "FMA:" in str:
      return self.expand_fma_id(str)
    elif "UBERON:" in str:
      return self.expand_uberon_id(str)
    elif "PCL:" in str:
      return self.expand_pcl_id(str)
    elif "CL:" in str:
      return self.expand_cl_id(str)
    elif "LMHA:" in str:
      return self.expand_lmha_id(str)
    elif "HGNC:" in str:
      return self.expand_hgnc_id(str)
    elif "ASCTB-TEMP:" in str:
      return self.expand_asctb_temp_id(str) 
    elif "DOI:" in str:
      return self.expand_doi(str)    
    else:
      return f"{self.ASCTB_TEMP_PREFIX}{str}"

  def expand_fma_id(self, str):
    fma_pattern = re.compile("FMA:", re.IGNORECASE)
    return fma_pattern.sub(self.FMA_PREFIX, str)

  def expand_uberon_id(self, str):
    uberon_pattern = re.compile("UBERON:", re.IGNORECASE)
    return uberon_pattern.sub(f"{self.OBO_PREFIX}UBERON_", str)

  def expand_pcl_id(self, str):
    pcl_pattern = re.compile("PCL:", re.IGNORECASE)
    return pcl_pattern.sub(f"{self.OBO_PREFIX}PCL_", str)
          
  def expand_cl_id(self, str):
    cl_pattern = re.compile("CL:", re.IGNORECASE)
    return cl_pattern.sub(f"{self.OBO_PREFIX}CL_", str)

  def expand_lmha_id(self, str):
    lmha_pattern = re.compile("LMHA:", re.IGNORECASE)
    return lmha_pattern.sub(f"{self.OBO_PREFIX}LMHA_", str)

  def expand_hgnc_id(self, str):
    hgnc_pattern = re.compile("HGNC:", re.IGNORECASE)
    return hgnc_pattern.sub(self.HGNC_PREFIX, str)
                  
  def expand_asctb_temp_id(self, str):
    asctb_temp_pattern = re.compile("ASCTB-TEMP:", re.IGNORECASE)
    return asctb_temp_pattern.sub(self.ASCTB_TEMP_PREFIX, str)
      
  def expand_doi(self, str):
    doi_pattern = re.compile("DOI:\\s*", re.IGNORECASE)
    return doi_pattern.sub(self.DOI_PREFIX, str)

In [5]:
BASE_URL = "https://asctb-api.herokuapp.com"
VERSION = "v2"
SHEETID = "1NMfu1bEGNFcTYTFT-jCao_lSbFD8n0ti630iIpRj-hw" # Kidney_v1.2 DRAFT
GID = "949267305"

In [6]:
url = f"{BASE_URL}/{VERSION}/{SHEETID}/{GID}"

In [7]:
response = get_json(url)

In [20]:
transformer = AsctbDataTransformer.new()
output = transformer.to_rdf(response);
print(output)

@prefix asctb-temp: <https://purl.org/ccf/ASCTB-TEMP_> .
@prefix ccf: <http://purl.org/ccf/> .
@prefix hgnc: <http://identifiers.org/hgnc/> .
@prefix obo: <http://purl.obolibrary.org/obo/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .

<https://doi.org/10.48539/HBM248.CBJV.556> a ccf:AsctbTable ;
    ccf:contains_records ( [ a ccf:AsctbRecord ;
                ccf:contains_anatomical_structures ( obo:UBERON_0002113 obo:UBERON_0002015 ) ;
                ccf:contains_biomarkers ( hgnc:3802 hgnc:1975 ) ;
                ccf:contains_cell_types [ rdf:rest () ] ;
                ccf:contains_gene_biomarkers ( hgnc:3802 hgnc:1975 ) ;
                ccf:contains_lipids_biomarkers [ rdf:rest () ] ;
                ccf:contains_meta_biomarkers [ rdf:rest () ] ;
                ccf:contains_prot_biomarkers [ rdf:rest () ] ;
                ccf:contains_protein_biomarkers [ rdf:rest () ] ;
                ccf:contains_references ( <http://doi.org/10.1371/journal.pone.0187118> )